In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [218]:
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",  # 모델 저장소 ID를 지정합니다.
    #repo_id = "mistralai/Mistral-7B-v0.1",  # 모델 저장소 ID를 지정합니다.    
    max_new_tokens=512,  
    temperature=0.01, # 샘플링 온도를 설정합니다. 값이 높을수록 더 다양한 출력을 생성합니다.
    callbacks=[StreamingStdOutCallbackHandler()],  # 콜백을 설정합니다.
    streaming=True,  # 스트리밍을 사용합니다.
   
)


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\cowbo\.cache\huggingface\token
Login successful


In [219]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("{country}의 수도에 대해서 한글로 알려줘")

In [220]:
chain = prompt | llm 

In [221]:
chain.invoke({"country":"대한민국"})


요

대한민국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로서 국내외적으로도 중요한 역할을 하고 있습니다. 서울은 한국의 역사와 문화가 풍부하여, 관광객들에게도 많은 관심을 받고 있습니다. 서울에는 유명한 관광지들이 많이 있으며, 그 중 일부는 궁전, 문화유적지, 공원, 미술관, 쇼핑몰 등이 있습니다. 또한 서울은 맛있는 음식도 많이 있어, 음식탐방도 서울에서 즐길 수 있습니다.</s>

'요\n\n대한민국의 수도는 서울입니다. 서울은 한국의 정치, 경제, 문화의 중심지로서 국내외적으로도 중요한 역할을 하고 있습니다. 서울은 한국의 역사와 문화가 풍부하여, 관광객들에게도 많은 관심을 받고 있습니다. 서울에는 유명한 관광지들이 많이 있으며, 그 중 일부는 궁전, 문화유적지, 공원, 미술관, 쇼핑몰 등이 있습니다. 또한 서울은 맛있는 음식도 많이 있어, 음식탐방도 서울에서 즐길 수 있습니다.</s>'

## Runnable

RunnablePassthrough

In [124]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

In [125]:
chain = {"country": RunnablePassthrough()} | prompt | llm 

In [ ]:
chain.invoke("미국")

RunnableParallel

In [222]:
prompt1 = PromptTemplate.from_template("ETF 상품인 {stock}의 분기별 배당월에 대해서 한글로 알려줘")
prompt2 = PromptTemplate.from_template("ETF 상품인 {stock}의 보유 비중 top 10 기업에 대해서 한글로 알려줘")

chain1 = {"stock": RunnablePassthrough()} | prompt1 | llm 
chain2 = {"stock": RunnablePassthrough()} | prompt2 | llm 



In [223]:
chain1.invoke("SCHD")

 주세요.

SCHD는 주요 업계 선도적인 회사들로 구성된 배당성 강화 전략 ETF입니다. 이 ETF는 매 분기 마지막 주말에 배당을 지불하며, 다음은 분기별 배당 지불 날짜입니다.

* 1분기: 3월 말
* 2분기: 6월 말
* 3분기: 9월 말
* 4분기: 12월 말

따라서, SCHD ETF는 매년 4번 배당을 지불하며, 각 분기 마지막 주말에 배당을 지불합니다.</s>

' 주세요.\n\nSCHD는 주요 업계 선도적인 회사들로 구성된 배당성 강화 전략 ETF입니다. 이 ETF는 매 분기 마지막 주말에 배당을 지불하며, 다음은 분기별 배당 지불 날짜입니다.\n\n* 1분기: 3월 말\n* 2분기: 6월 말\n* 3분기: 9월 말\n* 4분기: 12월 말\n\n따라서, SCHD ETF는 매년 4번 배당을 지불하며, 각 분기 마지막 주말에 배당을 지불합니다.</s>'

In [224]:
chain2.invoke("SCHD")

요

SCHD는 하이 다이�İ브İ드 디İ버İ펜스 전략을 기반으로 한 배당지수 ETF입니다. 이 ETF는 미국 주식시장의 상위 400개 회사 중 가장 안정적이고 지속적인 배당을 내는 회사들을 선택하여 투자합니다.

SCHD의 상위 10개 보유 기업은 다음과 같습니다.

1. Microsoft (MSFT)
2. Johnson & Johnson (JNJ)
3. JPMorgan Chase (JPM)
4. Visa (V)
5. Procter & Gamble (PG)
6. Coca-Cola (KO)
7. PepsiCo (PEP)
8. Verizon Communications (VZ)
9. The Home Depot (HD)
10. Realty Income Corporation (O)

이 기업들은 모두 미국 주요 산업의 선도적인 기업으로, 안정적인 수익성과 지속적인 배당을 보여줍니다.</s>

'요\n\nSCHD는 하이 다이�İ브İ드 디İ버İ펜스 전략을 기반으로 한 배당지수 ETF입니다. 이 ETF는 미국 주식시장의 상위 400개 회사 중 가장 안정적이고 지속적인 배당을 내는 회사들을 선택하여 투자합니다.\n\nSCHD의 상위 10개 보유 기업은 다음과 같습니다.\n\n1. Microsoft (MSFT)\n2. Johnson & Johnson (JNJ)\n3. JPMorgan Chase (JPM)\n4. Visa (V)\n5. Procter & Gamble (PG)\n6. Coca-Cola (KO)\n7. PepsiCo (PEP)\n8. Verizon Communications (VZ)\n9. The Home Depot (HD)\n10. Realty Income Corporation (O)\n\n이 기업들은 모두 미국 주요 산업의 선도적인 기업으로, 안정적인 수익성과 지속적인 배당을 보여줍니다.</s>'

In [225]:
combined_chain = RunnableParallel(a=chain1, b=chain2)


In [ ]:
combined_chain.invoke("SCHD")

In [227]:
def combine_text(text):
    return text["a"] + "," + text["b"]

In [ ]:
final_chain = combined_chain | {"info" : RunnableLambda(combine_text)} | PromptTemplate.from_template("{info}의 내용을 자연스럽게 연결후 한글로 작성해줘 \n ") | llm
final_chain.invoke("SCHD")